# Ejercicio 3 - Análisis de emociones en texto no-estructurado (NLP)

La idea es realizar un análisis de emociones desde comentarios en medios sociales, por medio de un modelo de clasificación supervisada, que entrega una de 3 posibles clases de emoción.

## Contexto: Análisis de texto de comentarios en medios sociales de diversos países (Chile, Arg, Mex, otros)

Este conjunto de datos generado en 2016 por [R:Solver](http://rsolver.com) y compartido parcialmente para este tipo de ejercicios, consiste en dos columnas: el texto del comentario, y una clasificación dentro de las tres alternativas o clases de emoción.

El gran objetivo final a resolver con este ejemplo, es analizar una porción de texto y lograr predecir la emoción correspondiente, dentro de 3 posibles clases:

*     Enojo
*     Sorpresa
*     Alegría

**CONSIDERACIÓN IMPORTANTE**

Se sabe que para lograr buenos resultados se requiere contar con muchos miles de ejemplos y en este ejercicio académico sólo se cuenta con pocos miles por cada clase. En la práctica se podrá apreciar que el desempeño general (*accuracy*) de los modelos es mediano-bajo (cerca del 50%). Esto, seguramente, podría mejorar recién al contar con varias decenas de miles de ejemplos adicionales a los actuales, lo que se explica por la gran diversidad de terminología encontrada en estos comentarios (son cerca de 20.000 términos o palabras diferentes en este corpus).

Por esta razón, el foco está en realizar análisis sobre estos desempeños y sacar conclusiones útiles, hasta cierto punto independiente de los *accuracies* correspondientes. Este ejercicio busca reforzar conceptos desde la perspectiva de quienes ya están adentrándose más en entender el funcionamiento de modelos de clasificación supervisada.

---

## Instrucciones Generales

Se deben contestar las preguntas que se indican en las secciones de "Preguntas", más adelante. Se puede recurrir a ejercicios de otras fuentes, así como al material de clases.

La entrega se realiza en forma de un informe en formato PDF, utilizando la plantilla de informe que está en http://dcc.rsolver.com/dcc/docs/InformeActividad.docx

Esta entrega se puede subir por un único miembro del grupo a la tarea o registro en el portal del curso.

En caso de no poder subirlo por esa vía o que no esté habilitado, se puede usar el buzón alternativo: http://aiker.rsolver.com/aiker/DocUpload.aspx (*)

(*) Si y solo si hay problemas en la carga, enviar el PDF a rsandova@ing.puc.cl y cc: ayudante@aiker.ai


## Paso 1: Instalar librerías de modelos de clasificación

In [1]:
#install.packages('e1071')
#install.packages('caret')
#install.packages('caTools')
#install.packages('tm')
#install.packages('rpart')
#install.packages('SnowballC')
#install.packages('nnet')

Installing package into ‘/home/nestorprr/R/x86_64-pc-linux-gnu-library’
(as ‘lib’ is unspecified)

Installing package into ‘/home/nestorprr/R/x86_64-pc-linux-gnu-library’
(as ‘lib’ is unspecified)

Installing package into ‘/home/nestorprr/R/x86_64-pc-linux-gnu-library’
(as ‘lib’ is unspecified)

Installing package into ‘/home/nestorprr/R/x86_64-pc-linux-gnu-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘NLP’, ‘slam’


Installing package into ‘/home/nestorprr/R/x86_64-pc-linux-gnu-library’
(as ‘lib’ is unspecified)

Installing package into ‘/home/nestorprr/R/x86_64-pc-linux-gnu-library’
(as ‘lib’ is unspecified)

Installing package into ‘/home/nestorprr/R/x86_64-pc-linux-gnu-library’
(as ‘lib’ is unspecified)



In [1]:
library(tm)
library(SnowballC)
library(caTools)
library(caret)
library(rpart)
library(nnet)

Loading required package: NLP

Loading required package: ggplot2


Attaching package: ‘ggplot2’


The following object is masked from ‘package:NLP’:

    annotate


Loading required package: lattice



## Paso 2: Carga de los datos

La siguiente celda de código carga los datos desde la URL de origen y luego muestra un encabezado con las primeras filas del dataset, para demostrar la disposición y ejemplos de los datos.

**Nótese que hay un desbalance entre los ejemplos de cada una de las 3 clases, pero NO se espera ni se pide modificar esto, lo cual queda a criterio de los alumnos experimentar con re-balancear las clases y ver su desempeño.**


In [2]:
# Se declara la URL de dónde obtener los datos
theUrlMain <- "http://RAlize.RSolver.com/RAlize/data/small_emotion_sample_2016.csv"

In [3]:
# Se declaran los nombres de las columnas
columnas <- c("texto","emoción")

# Se cargan datos principales a una estructura (commentsdataset), asignando nombres de atributos a las columnas
comments.dataset.raw <- read.csv(file = theUrlMain, header = FALSE, sep = ";", col.names=columnas, skipNul = TRUE)

# Conteo de ejemplos de cada clase.
# Al ver su dimensión, se puede apreciar si están o no balanceados,
# pero no se espera en esta ocasión trabajar en balanceo de clases.
data.enojo <- comments.dataset.raw[comments.dataset.raw$emoción == 'enojo',]
data.sorpresa <- comments.dataset.raw[comments.dataset.raw$emoción == 'sorpresa',]
data.alegria <- comments.dataset.raw[comments.dataset.raw$emoción == 'alegria',]
cat("\nEjemplos Enojo:     ", dim(data.enojo))
cat("\nEjemplos Sorpresa:  ", dim(data.sorpresa))
cat("\nEjemplos Alegría:   ", dim(data.alegria))

commentsdataset <- comments.dataset.raw
dim(commentsdataset)
head(commentsdataset, 20)


Ejemplos Enojo:      3054 2
Ejemplos Sorpresa:   1085 2
Ejemplos Alegría:    2762 2

[1] 6901    2

,texto,emoción
,<chr>,<chr>
1,!FAVOR REVISAR DICCIONARIO !,enojo
2,?El Dodge Challenger cuantos kilometros por litro dara? Esa es mi consulta,sorpresa
3,"el trasero trailer de la version en carne y hueso , esos redactores merecen el premio redactor de excelencia jajaj",alegria
4,No sera un Peruano o Boliviano que lo hace para que sancionen a Chile ? Hay que acordarse de la guerra mediatica y todos los reclamos de parte de Evo A ese Evo tambien se le sospecha que fue quien dio la orden del reclamo boliviano por los canticos en el estadio,enojo
5,Y Renaca y Concon para levantar carpas en la playa,sorpresa
6,"Yo siempre compro por internet pero cada vez que hay Cyber Monday termino decepcionado, es mas la publicidad que reales ofertas",enojo
7,! Arjonaaaa !!!!! !Este otro ano vas fijo !!!!,sorpresa
8,!Partidazo!,alegria
9,"Medicion de pobreza, mas moderna y transparente: En enero de 2014 la Comision para la Medicion de la Pobreza, planteo un conjunto de propuestas de actualizacion de la metodologia de medicion de pobreza por ingresos, asi como la incorporacion de un enfoque de medicion multidimensional Sus sugerencias fueron recibidas por el gobierno de la Presidenta Bachelet y analizadas en detalle durante 2014 por una Mesa Tecnica Interinstitucional, conformada por el Ministerio de Desarrollo Social (MDS) y el Instituto Nacional de Estadisticas (INE), con el apoyo tecnico de Comision Economica para America Latina (CEPAL) En el ambito de la pobreza multidimensional, se conto ademas con la asesoria permanente de la Iniciativa para la Pobreza y el Desarrollo Humano de la Universidad de Oxford (OPHI, por su sigla en ingles)",sorpresa


## Ejercicio 1: Entender el efecto de la normalización de texto

Las técnicas de normalización de texto, cuando tienen que realizar análisis del texto leído, deben buscar la simplificación del texto para poder trabajar sobre un universo de términos más simple y acotado. Esto puede considerar eliminar tildes, evitar palabras comunes, usar la raíz de múltiples términos, entre otros.

A continuación se aplican algunas de las técnicas más frecuentes, permitiendo ver el efecto (potencialmente positivo) de cada una de ellas.

**Pregunta 1.1 (1 punto)**: ¿Cuánto mejora el rendimiento de los modelos de análisis de sentimiento (implementados en la siguiente sección) al agregar las técnicas de normalización? La idea es comparar el rendimiento con y sin las técnicas de normalización, analizando cuantitativamente las posibles mejoras en cada uno de los dos modelos (a nivel de sus indicadores de desempeño).

**Pregunta 1.2 (0,5 puntos)**: ¿Cuál es el modelo que mejora más? Analice y describa qué característcas de este modelo justifican ser el que mejora con las técnicas de normalización, más que el otro.

Aquí se recomienda registrar el desempeño de los modelos habiendo ejecutado con y sin las técnicas de normalización. Para verificar el desempeño SIN la normalización se puede eliminar las 5 instrucciones o técnicas que están el código como Bloque 1 (se recomienda comentar la línea de código respectivo con un símbolo #).

Se debe comparar el desempeño de ambos modelos, con y sin las normalizaciones de texto, de modo de ver en cuál de ellos tiene más efecto. Para efectos prácticos, se propone realizar la modificación en el BLOQUE 1 y correr todas las secciones de código en adelante, para ver el rendimiento comparado.  

Luego de contestar estas preguntas, continue con el ejercicio dejando activas las técnicas de normalización en adelante.

In [4]:
# Construye el Corpus: el universo de texto que se usará para entrenar los modelos.
corpus.original <- Corpus(VectorSource(commentsdataset$texto))

# Se selecciona y muestra (sin normalizar) un comentario de ejemplo aleatorio dentro del corpus
random_index <- floor(runif(1, min=0, max=length(corpus.original)))
content(corpus.original[[random_index]])

################################
# NORMALIZACIÓN DEL TEXTO
# EJERCICIO: probar comentando (anteponiendo #) estas acciones,
# para ver cuánto es el efecto en el rendimiento del modelo de clasificación
################################
# Se saca una copia ('corpus') de trabajo, para no alterar el original
corpus <- corpus.original

## BLOQUE 1
# Se pasan todas las palabras a minúsculas
corpus <- tm_map(corpus, tolower)
# Se eliminan todos los signos de puntuación
corpus <- tm_map(corpus, removePunctuation)
# Se eliminan todos los números
corpus <- tm_map(corpus, removeNumbers)
# Se eliminan las stop words (palabras comunes, irrelevantes)
corpus <- tm_map(corpus, removeWords, c(stopwords("spanish")))
# Se lleva cada palabra a su raíz (stemming)
corpus <- tm_map(corpus, stemDocument)


# Se muestra el mismo ejemplo aleatorio, pero en texto normalizado
content(corpus[[random_index]])


[1] "AFP: el poder impotente Ciper - Columna de DMatamala - AFP: el poder impotente"

Warning message in tm_map.SimpleCorpus(corpus, tolower):
“transformation drops documents”
Warning message in tm_map.SimpleCorpus(corpus, removePunctuation):
“transformation drops documents”
Warning message in tm_map.SimpleCorpus(corpus, removeNumbers):
“transformation drops documents”
Warning message in tm_map.SimpleCorpus(corpus, removeWords, c(stopwords("spanish"))):
“transformation drops documents”
Warning message in tm_map.SimpleCorpus(corpus, stemDocument):
“transformation drops documents”


[1] "afp poder impotent ciper columna dmatamala afp poder impotent"

## Ejercicio 2: Construcción de un Vocabulario con Términos Significativos

Los clasificadores reciben un X de entrada de una dimensión fija. Por lo tanto los X de este ejemplo de análisis de texto, comentarios de cantidad variable de palabras, no se pueden usar tal cual vienen (como lista de cantidad variables de términos).

Por ello, la siguiente porción de código transforma los comentarios de texto variable en un vector de ocurrencia de palabras referenciando un vocabulario, el cual se construye referenciando todas las palabras distintas (ya normalizadas) del Corpus. Esto se traduce en que un comentario que tiene la expresión ".. resultados excelentes ... ", tendría una intersección con el comentario "... excelente como resultó ..." y por ello podrían ser interpretadas en forma equivalente.

Pero el desafío de esta vectorización en base a un vocabulario es la **cantidad de dimensiones** (cantidad de palabras diferentes). Un vocabulario perfectamente puede tener varios miles de palabras diferentes (ver resultado en 2.A), entonces la dimensión del vector X es de esos varios miles.

Esto motiva a reducir la dimensionalidad del problema (el tamaño del vocabulario) al reconocer cuáles son los términos más relevantes. Esto se hace con removeSparseTerm() y un umbral alto (0.995 reduce la gran cantidad de términos que tienen al menos un 99.5% de 0s en la columna), como se ve en en 2.B. Mientras mayor es el número, mayor cantidad de términos ocasionales o esporádicos se conservan y el vocabulario queda más grande. En otras palabras, mientras menor es el número, mayor es la exigencia para un término de ser considerado valioso y quedar como parte del dataset. (https://www.rdocumentation.org/packages/tm/versions/0.7-8/topics/removeSparseTerms)

La pregunta específica a responder cuando se implementa un modelo es: ¿cuántos son los términos a considerar en el vocabulario que sean los mejores representativos del universo de diversas palabras, para maximizar el desempeño del modelo de clasificación? Eso se responde buscando un factor de removeSparseTerms() que logre ese mejor desempeño.

**Pregunta 2.1 (1 punto)** ¿Cuál es el valor usado en removeSparseTerm() para lograr mejores resultados en los modelos de clasificación? Realice una comparación de 3 valores: 0.990, 0.995, 0.998.

**Pregunta 2.2 (1 punto)** ¿Cómo se interpreta en este caso que mejore o empeore con más y a veces con menos cantidad de palabras (o columnas del dataset)?

Para ambas preguntas se debe comparar los resultados del Árbol de Decisión y de la Red Neuronal (utilizando parámetros de ejecución más liviana - ej: size=20 y maxit=30).


In [5]:
#######################################################
# Indexación de términos: creación de un Vocabulario
#######################################################

# Primero una matriz de ocurrencia de términos o palabras (DTM: Document-term matrix).
# Las filas son los comentarios y las columnas son las palabras diferentes encontradas (varios miles).
termMatrix <- DocumentTermMatrix(corpus)
dim(termMatrix)   # Resultado 2.A -> Las columnas son todos los términos diferentes encontrados en corpus

# Entonces, se eliminan las palabras menos relevantes (sparse terms: términos dispersos)
# lo que resulta en una reducción dimensional (potencialmente grande)
# Ejercicio 2.1: ¿qué factor de eliminación (en rango 0.99x) da mejores resultados?
# Probar con 3 combinaciones: 0.990, 0.995, 0.998.
# Ejercicio 2.2: ¿qué implica o cómo se interpreta ese cambio de valor y su efecto en el dataset?
termMatrixLight <- removeSparseTerms(termMatrix, 0.995)
dim(termMatrixLight)  # Resultado 2.B -> Se puede ver que se reduce significativamente la cantidad de columnas (palabras)

# Re-formatea como un DataFrame
corpus.procesado <- as.data.frame(as.matrix(termMatrixLight))

# Se construye el dataset para entrenamiento, recuperando el comentario original, sin procesar
# agregando la columna "emoción"
corpus.procesado$emoción <- as.factor(commentsdataset$emoción)

# Se muestra el vocabulario y su tamaño
dim(corpus.procesado)

[1]  6901 16848

[1] 6901  195

[1] 6901  196

## Paso 3: Ejecución modelos de predicción según conjuntos de entrenamiento y validación

En este caso, el dataset de ejemplos etiquetados se divide en dos (Hold-out) para entrenar y validar con conjuntos disjuntos.

No se necesita modificar esta sección, aunque es válido y potencialmente interesante revisar diferentes valores de proporción entrenamiento/validación.


In [6]:
################################################################
# Versión simple para crear conjuntos de entrenamiento y de test.
# Se puede dejar la proporción original de 0.70,
# pero se puede cambiar si se piensa que puede hacer una diferencia.
ratio <- sample(nrow(corpus.procesado),nrow(corpus.procesado)*0.70)
training.set = corpus.procesado[ratio,]
testing.set  = corpus.procesado[-ratio,]

cat("Dimensiones Entrenamiento/Test")
dim(training.set)
dim(testing.set)

head(testing.set, 10)

Dimensiones Entrenamiento/Test

[1] 4830  196

[1] 2071  196

,favor,cuanto,chile,dio,hace,part,sera,tambien,cada,mas,⋯,via,falabella,…,transantiago,artifici,monsanto,transgenico,camilavallejo,gustado,emoción
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
2,0,1,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,sorpresa
3,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,alegria
6,0,0,0,0,0,0,0,0,1,1,⋯,0,0,0,0,0,0,0,0,0,enojo
12,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,sorpresa
20,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,sorpresa
21,0,0,0,0,0,0,1,0,0,0,⋯,0,0,0,0,0,0,0,0,0,enojo
22,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,enojo
23,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,sorpresa
30,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,sorpresa


## Ejercicio 3: Interpretación de desempeño de modelos de clasificación de referencia

Habiendo definido y establecido los conjuntos de entrenamiento y de test, a continuación se ejecutan dos diferentes modelos de clasificación: Árbol de Decisión y más abajo una Red Neuronal (NNET). Cada uno obtiene su resultado, mostrando sus indicadores de desempeño.

Nótese que ninguno de los 2 modelos da resultados significativamente buenos o mucho mejor que el otro, lo cual se debe a la inherente complejidad de los casos de lenguaje natural y contar sólo con unos pocos miles de ejemplos (en total son cerca de 7.000 ejemplos, divididos en 3 clases, lo cual no alcanza a reflejar la real diversidad de palabras asociadas a cada una de las clases, cosa que se empieza a lograr desde las decenas de miles de ejemplos por clase).

Sin embargo, hay algunas observaciones que salen de los resultados de desempeño, además del hecho de que la red neuronal toma cerca de 10x el tiempo del Árbol de Decisión. De estas observaciones, se pide responder lo siguiente.

**Pregunta 3.1 (1 punto)** Considere cambiar los parámetros de funcionamiento de la red neuronal (size, maxit) buscando su mejor desempeño e indique los valores de estos dos parámetros que resultó en mejores resultados.

**Pregunta 3.2 (0,7 punto)** Teniendo este resultado se ve que de los dos modelos, hay uno con *accuracy* mejor que el otro, pero en forma más completa y precisa ¿cuál de los dos se puede considerar el mejor modelo al mirar su desempeño más completo y por qué? Nótese que la tabla de resultados tiene una serie de indicadores diferentes por cada una de las 3 clases, que sirven para entender cómo se comporta el modelo en una correcta clasificación.

**Pregunta 3.3 (0,8 punto)** Se puede observar que el Árbol de Decisión tiene un muy mal desempeño en reconocer una de esas clases. ¿Cuál y por qué podría darse esto? (En otras palabras, ¿qué característica de este modelo puede provocar que en este caso una de las clases no sea factible de predecir?


**Árbol de Decisión**

In [7]:
DT_model <- rpart(emoción ~ ., data=training.set, method="class", minbucket=20)

DT_predictTraining <- predict(DT_model, training.set, type = "class")
DT_predictTesting <- predict(DT_model, testing.set, type = "class")
cat("\n\n************* Resultados Árbol de Decisión - Testing *************\n")
confusionMatrix(DT_predictTesting, as.factor(testing.set$emoción))



************* Resultados Árbol de Decisión - Testing *************


Confusion Matrix and Statistics

          Reference
Prediction alegria enojo sorpresa
  alegria       22     0        2
  enojo        765   954      328
  sorpresa       0     0        0

Overall Statistics
                                         
               Accuracy : 0.4713         
                 95% CI : (0.4496, 0.493)
    No Information Rate : 0.4606         
    P-Value [Acc > NIR] : 0.1716         
                                         
                  Kappa : 0.0214         
                                         
 Mcnemar's Test P-Value : <2e-16         

Statistics by Class:

                     Class: alegria Class: enojo Class: sorpresa
Sensitivity                 0.02795      1.00000          0.0000
Specificity                 0.99844      0.02149          1.0000
Pos Pred Value              0.91667      0.46605             NaN
Neg Pred Value              0.62628      1.00000          0.8407
Prevalence                  0.38001      0.46065          0.1593


**Red Neuronal**

Nótese que toma varios minutos la ejecución de entrenamiento y evaluación de esta red, por lo que se recomienda hacer uso estratégico de los cambios en la sección/ejercicio 1 y 2 anteriores antes de probar.

In [8]:
# Parámetros de nnet():
#     size: moverse entre 5 y 40. Es la cantidad de nodos de la única capa intermedia.
#           Sin embargo, nótese que según el tamaño del vocabulario,
#           valores altos de size pueden resultar en error de ejecución.
#           En estos casos, sólo queda reducir el valor de size hasta que MaxNWts quede en menos de 30.000.
#     maxit: 20 para pruebas cortas, hasta más de 100 (varios minutos de ejecución)
#     MaxNWts: cantidad tope de arcos de conexión. 30.000 se mantiene dentro del tope de RAM,
#              pero puede requerir varios minutos de ejecución.
cat("Entrenando Red Neuronal\n")
NN.model <- nnet(as.factor(emoción) ~ ., data=training.set, size=100, maxit=50, MaxNWts=30000)
NN.predict <- predict(NN.model, testing.set, type="class")

cat("Resultados Red Neuronal\n")
x <- testing.set[, 1:dim(corpus.procesado)[2]-1]
y <- testing.set[, dim(corpus.procesado)[2]]
y.predicted <- predict(NN.model, x, type = 'class')
confusionMatrix(as.factor(y.predicted), y)


Entrenando Red Neuronal
# weights:  19903
initial  value 14703.740138 
iter  10 value 4855.141350
iter  20 value 4193.552633
iter  30 value 3463.320006
iter  40 value 2870.985334
iter  50 value 2365.020134
final  value 2365.020134 
stopped after 50 iterations
Resultados Red Neuronal


Confusion Matrix and Statistics

          Reference
Prediction alegria enojo sorpresa
  alegria      412   392      119
  enojo        299   455      138
  sorpresa      76   107       73

Overall Statistics
                                          
               Accuracy : 0.4539          
                 95% CI : (0.4323, 0.4756)
    No Information Rate : 0.4606          
    P-Value [Acc > NIR] : 0.7385          
                                          
                  Kappa : 0.1084          
                                          
 Mcnemar's Test P-Value : 9.907e-06       

Statistics by Class:

                     Class: alegria Class: enojo Class: sorpresa
Sensitivity                  0.5235       0.4769         0.22121
Specificity                  0.6020       0.6088         0.89489
Pos Pred Value               0.4464       0.5101         0.28516
Neg Pred Value               0.6733       0.5768         0.85840
Prevalence                   0.3800       0.4606        